# Agent Lab Draft 3 - Examples

In [1]:
# Import necessary packages

%matplotlib inline

import torch

from autogen import ConversableAgent, AssistantAgent
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Mostly just code from lab 3 packaged into functions
import lab3code as lab3

## Define Model

In [2]:
class mnistClassification(torch.nn.Module):
    
    def __init__(self, input_dim, output_dim): # Feel free to add hidden_dim as parameters here
        super(mnistClassification, self).__init__()
        
        self.description = "Simple FCN with 2 layers, ReLU activation, and dropout"
        
        #self.layerconv1 = torch.nn.Conv1d(in_channels=1, out_channels=1, kernel_size=5, stride=2, padding=2)
        self.layer1 = torch.nn.Linear(input_dim, 200) # First layer
        self.layer2 = torch.nn.Linear(200, output_dim) # Second layer
        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(p=0.2) # Dropout layer with 20% dropout rate
        
    def forward(self, x):
        
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = torch.nn.functional.softmax(x, dim=1)
        
        return x

## Agents

### DatasetLoader

In [3]:
class DatasetLoaderAgent(AssistantAgent):
    """
    An agent that specializes in loading and preprocessing datasets.
    """
    def __init__(self, name="DatasetLoader", **kwargs):
        self.available_datasets = ["mnist", "solar_flare"]
        system_message = (
            "I am a dataset loading assistant. I can load and preprocess various datasets for machine learning tasks. "
            f"Currently I support: {', '.join(self.available_datasets)}. "
        )
        super().__init__(name=name, system_message=system_message, **kwargs)
        
        # Store loaded datasets
        self._loaded_datasets = {}
    
    def get_available_datasets(self):
        """Return a list of available datasets."""
        return self.available_datasets
    
    def load_dataset(self, dataset_name):
        """
        Load and preprocess a dataset.
        
        Args:
            dataset_name (str): Name of the dataset to load
            
        Returns:
            dict: Information about the loaded dataset and the data itself
        """
        try:
            # Load the dataset using the existing function
            train_features, train_targets, val_features, val_targets, test_features, test_targets = lab3.load_dataset(dataset_name)
            
            # Store the dataset
            self._loaded_datasets[dataset_name] = {
                "train_features": train_features,
                "train_targets": train_targets,
                "validation_features": val_features,
                "validation_targets": val_targets,
                "test_features": test_features,
                "test_targets": test_targets,
            }
            
            # Return information about the loaded dataset
            return {
                "status": "success",
                "dataset_name": dataset_name,
                "train_samples": train_features.shape[0],
                "validation_samples": val_features.shape[0],
                "test_samples": test_features.shape[0],
                "feature_dim": train_features.shape[1]
            }
        except Exception as e:
            return {
                "status": "error",
                "message": f"Failed to load dataset '{dataset_name}': {str(e)}"
            }
    
    def get_dataset(self, dataset_name):
        """
        Retrieve a previously loaded dataset.
        
        Args:
            dataset_name (str): Name of the dataset to retrieve
            
        Returns:
            dict: The dataset components or None if not found
        """
        return self._loaded_datasets.get(dataset_name)

### InterfaceAgent

In [4]:
# Load the model
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32)
model.to("cuda" if torch.cuda.is_available() else "cpu")

# Create a simple text-generation pipeline
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=200,
    do_sample=True,
    temperature=0.4,
)

# Wrapper function for the local model pipeline
def local_model_generate(prompt):
    output = llm_pipeline(prompt)[0]["generated_text"]
    return output

Device set to use cpu


In [5]:
# Agent definition
class InterfaceAgent(ConversableAgent):
    def __init__(self, name, **kwargs):
        super().__init__(name, **kwargs)
        
        self.dataset_loader_agent = None
        self.model_trainer_agent = None

    def set_dataset_loader_agent(self, agent):
        self.dataset_loader_agent = agent
        
    def set_model_trainer_agent(self, agent):
        self.model_trainer_agent = agent
    
    def generate_reply(self, messages):
        """
        There is a lot of obtuse text parsing and such here. I would describe the code as "technically functional".
        This is becasue the LLM is so limited. With a better LLM, and/or ideally one trained explicitly for 
        agentic implementation, this would be much cleaner and more flexible. 
        C.f. retrieval augmented generation, etc.
        """
        # Extract the latest user message
        user_message = messages[-1]["content"]
        
        # Check if the message is a "get dataset" command
        # get dataset <dataset_name>
        if "get dataset" in user_message:
            dataset_name = user_message.replace("get dataset", "").strip()
            
            # Use the DatasetLoaderAgent to load the dataset
            result = self.dataset_loader_agent.load_dataset(dataset_name)

            # Check if the dataset was successfully loaded
            if result["status"] == "success":
                response_text = (
                    f"Successfully loaded dataset '{result["dataset_name"]}'.\n"
                    f"Training samples: {result['train_samples']}, "
                    f"Validation samples: {result['validation_samples']}, "
                    f"Test samples: {result['test_samples']}, "
                    f"Feature dimension: {result['feature_dim']}."
                )
            else:
                response_text = f"Failed to load dataset '{dataset_name}': {result['message']}"
        
        # Check if the message is a "query" command
        # query <dataset_name>
        elif "query" in user_message:
            query = user_message.replace("query", "").strip()
            
            # Use the DatasetLoaderAgent to retrieve the dataset
            # Assuming the query is in the format "query <dataset_name>"
            # In principle, this is where you could have an LLM parse the command
            dataset_name = query.split()[0]
            dataset = self.dataset_loader_agent.get_dataset(dataset_name)
    
            if dataset:
                # Prepare a prompt with dataset information
                prompt = (
                    f"The dataset '{dataset_name}' has been loaded. "
                    f"The first two rows of the training features are:\n"
                    f"{dataset['train_features'][:2]}.\n"
                    "Short, concise description of the dataset:\n"
                )
                # Generate a description using the local model
                response_text = local_model_generate(prompt)
            else:
                response_text = f"Dataset '{dataset_name}' is not loaded or does not exist."
                
            
        elif "train" in user_message:
            # Should be of the form "train <dataset_name> <model_type>"
            # Again, this is where a specialized LLM would be really useful
            # Model type
            parts = user_message.split()
            dataset_name = parts[1]
            
            # Use the ModelTrainerAgent to train the model
            if self.model_trainer_agent is not None:
                # If the user specified an available model type, use that
                result = self.model_trainer_agent.train_model(dataset_name)
                
                if result["status"] == "success":
                    response_text = f"Successfully trained the model."
                else:
                    response_text = f"Failed to train the model: {result['response']}"
            else:
                response_text = "Model trainer agent is not set."
        
        # No special command, just a normal message. Can just use the model as a chatbot
        else:
            response_text = local_model_generate(user_message)
        
        # Return the response in the Autogen format
        return {"role": "assistant", "content": response_text}

## Load a dataset

Have the agents load a dataset and describe it

In [6]:
# Create instances of the agents
dataset_loader_agent = DatasetLoaderAgent(name="DatasetLoader")
local_agent = InterfaceAgent(name="LocalAgent")
local_agent.set_dataset_loader_agent(dataset_loader_agent)

# Simulate a conversation
messages = [{"role": "user", "content": "get dataset mnist"}]

# InterfaceAgent processes the first message
response = local_agent.generate_reply(messages)
print(response["content"])
print()

# Add a second message to the conversation
messages.append({"role": "user", "content": "query mnist"})

# InterfaceAgent processes the second message
response = local_agent.generate_reply(messages)
print(response["content"])
print()

# You can also use the model as a chatbot
messages.append({"role": "user", "content": "Why don't whales have feet?"})
response = local_agent.generate_reply(messages)
print(response["content"])
print()

# Note that the output is not reliable at all since the model is tiny. Sometimes it's surprisingly good though

Successfully loaded dataset 'mnist'.
Training samples: 800, Validation samples: 200, Test samples: 100, Feature dimension: 784.

The dataset 'mnist' has been loaded. The first two rows of the training features are:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]].
Short, concise description of the dataset:
The MNIST dataset contains 60,000 28x28 grayscale images of handwritten digits. The dataset is split into a training set of 60,000 images and a test set of 10,000 images. The training set is used to train the neural network, while the test set is used to evaluate the performance of the network. The dataset is used in many machine learning and deep learning applications, including image classification, handwritten digit recognition, and image denoising.

4. Preprocessing the dataset:
Load the dataset using the `load_data` function. The function takes the path to the dataset as an argument and returns a pandas DataFrame containing the training and test sets.

```python
import pandas 

## Model Trainer

In [7]:
class ModelTrainerAgent(ConversableAgent):
    """
    An agent that selects, creates, and trains a classification model based on the user's query.
    """
    def __init__(self, name="ModelTrainer", dataset_loader_agent=None, **kwargs):
        super().__init__(name=name, **kwargs)
        self.dataset_loader_agent = dataset_loader_agent
        
        self.model, self.train_loss_list, self.val_accuracy_list, self.test_accuracy = None, None, None, None
        
        self.available_models = ["linear"]
        

    def train_model(self, dataset_name):
        """
        Parse the query, select the model, and train it on the dataset.
        """
        try:
            model_type = "linear"

            # Retrieve the dataset from the DatasetLoaderAgent
            dataset = self.dataset_loader_agent.get_dataset(dataset_name)
            if not dataset:
                return f"Dataset '{dataset_name}' is not loaded or does not exist."

            # Extract dataset components
            train_features = dataset["train_features"]
            train_targets = dataset["train_targets"]
            val_features = dataset["validation_features"]
            val_targets = dataset["validation_targets"]
            test_features = dataset["test_features"]
            test_targets = dataset["test_targets"]

            # Select the model based on the model type
            if len(train_features.shape) == 3:
                indim = train_features.shape[1]*train_features.shape[2]
            else:
                indim = train_features.shape[1]
            
            # Training printing
            print(f"Training model of type '{model_type}' on dataset '{dataset_name}' with input dimension {indim}.")
            print(f"Training features shape: {train_features.shape}")
            print(f"Training targets shape: {train_targets.shape}")
            
            
            model = mnistClassification(input_dim=indim, output_dim=10)

            # Train the model
            self.model, self.train_loss_list, self.val_accuracy_list, self.test_accuracy = lab3.train_model(
                model, train_features, train_targets, val_features, val_targets,
                test_features=test_features, test_targets=test_targets
            )

            # Summarize the training results
            response = (
                f"Model '{model_type}' trained successfully on dataset '{dataset_name}'.\n"
                f"Final validation accuracy: {self.val_accuracy_list[-1]:.4f}\n"
                f"Test accuracy: {self.test_accuracy:.4f}"
            )
            print("Training response:", response)
            
            result = {
                "role": "assistant",
                "status": "success",
                "model": self.model,
                "train_loss_list": self.train_loss_list,
                "val_accuracy_list": self.val_accuracy_list,
                "test_accuracy": self.test_accuracy,
                "response": response
            }
            
            return result
        
        except Exception as e:
            return {"status": "error", "response": f"An error occurred during training: {str(e)}"}

In [8]:
# Create instances of the agents
dataset_loader_agent = DatasetLoaderAgent(name="DatasetLoader")
model_trainer_agent = ModelTrainerAgent(name="ModelTrainer", dataset_loader_agent=dataset_loader_agent)
local_agent = InterfaceAgent(name="LocalAgent")

local_agent.set_dataset_loader_agent(dataset_loader_agent)
local_agent.set_model_trainer_agent(model_trainer_agent)

# Simulate a conversation
messages = [{"role": "user", "content": "get dataset mnist"}]
response = local_agent.generate_reply(messages)
print(response["content"])
print()

# Add a training command
# "random" just to force the agents to talk to each other
messages.append({"role": "user", "content": "train mnist"})
response = local_agent.generate_reply(messages)
print(response["content"])
print()

# And you now have a trained model!
print(model_trainer_agent.model)

Successfully loaded dataset 'mnist'.
Training samples: 800, Validation samples: 200, Test samples: 100, Feature dimension: 784.

Training model of type 'linear' on dataset 'mnist' with input dimension 784.
Training features shape: (800, 784)
Training targets shape: (800,)


100%|██████████| 80/80 [00:01<00:00, 79.26it/s]

Training response: Model 'linear' trained successfully on dataset 'mnist'.
Final validation accuracy: 0.9150
Test accuracy: 0.8700
Successfully trained the model.

mnistClassification(
  (layer1): Linear(in_features=784, out_features=200, bias=True)
  (layer2): Linear(in_features=200, out_features=10, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
)
